In [1]:
import numpy as np

from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths
from src.vrp_study.data_loader import parse_data

In [2]:
PRINT_LOG = True

In [3]:
benchmark_type = 'pdp_400'
name = 'LC1_4_9.txt'

In [4]:
depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/{benchmark_type}/{name}')

In [5]:
p2coordinates = {}
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [6]:
len(cargos)

209

In [7]:

from src.vrp_study.configs import ModelConfig

routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

routing_manager = routing_manager.build()

In [8]:
from src.vrp_study.pdptw_model.solution_builder import SolutionBuilder

sol = find_optimal_paths(routing_manager,SolutionBuilder())

2025-08-11 15:10:15.689 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (1, 32.074999999999996)
2025-08-11 15:10:15.694 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (1, 16.0875)
2025-08-11 15:10:15.699 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (1, 8.09375)
2025-08-11 15:10:15.705 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (1, 4.096875)
2025-08-11 15:10:15.714 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (1, 2.0984374999999997)
2025-08-11 15:10:15.726 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (5, 1.09921875)
2025-08-11 15:10:15.736 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (8, 0.599609375)
2025-08-11 15:10:15.745 | INFO     | src.vrp_study.pdptw_model.solution_builder:get_initial_solution:120 - (15, 0.3498046875)
2025

  0%|          | 0/32 [00:00<?, ?it/s]

2025-08-11 15:10:15.776 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 49
2025-08-11 15:10:15.777 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-11 15:10:15.781 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-11 15:10:15.783 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-11 15:10:15.784 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-11 15:10:15.786 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-11 15:10:15.787 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-11 15:10:15.805 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-11

  0%|          | 0/1 [00:00<?, ?it/s]

2025-08-11 15:10:20.172 | INFO     | vrp_study.pdptw_model.routing_model:find_optimal_paths:443 - problem size: 3
2025-08-11 15:10:20.174 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:378 - Начало создания модели
2025-08-11 15:10:20.176 | INFO     | vrp_study.pdptw_model.routing_model:add_distance_dimension:299 - Добавление размерности для расстояния
2025-08-11 15:10:20.178 | INFO     | vrp_study.pdptw_model.routing_model:add_count_dimension:335 - Добавление размерности для расстояния
2025-08-11 15:10:20.179 | INFO     | vrp_study.pdptw_model.routing_model:add_pick_up_and_delivery:269 - Добавление ограничения для порядка доставки
2025-08-11 15:10:20.180 | INFO     | vrp_study.pdptw_model.routing_model:add_time_window:194 - add time
2025-08-11 15:10:20.182 | INFO     | vrp_study.pdptw_model.routing_model:add_mass_constraint:177 - Добавление ограничений для массы
2025-08-11 15:10:20.187 | INFO     | vrp_study.pdptw_model.routing_model:do_solve:407 - Начало решения
2025-08-11 

In [9]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        # a = routing_manager.nodes()[s[i]]
        # b = routing_manager.nodes()[s[i+1]]

        l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in
                range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l

7894.027885916032